In [ ]:
from IPython.core.display import HTML, display
display(HTML('<style>.container { width:100%; !important } </style>'))

In [ ]:
!pip install import_ipynb==0.1.3
!pip install chess

In [ ]:
import chess

import import_ipynb
import Game
from MinimaxAlgorithm import make_move_minimax, make_move_minimax_memoization
from RandomAlgorithm import make_random_move
from AlphaBetaAlgorithm import make_move_alphabeta, make_move_alphabeta_iterative_deepening




In [ ]:
# Simple manual test to make sure evaluate_move didn't break. Feel free to remove
#game = Game.Game(make_move_algorithm_black=make_move_alphabeta)
#game.play()

In [ ]:
# Random
# Minimax
    # MinimaxMemo
# AlphaBeta
    # AlphaBeta_IterativeDeepening

#random_minimax = Game.Game(make_random_move, make_move_minimax)
#random_alpha = Game.Game(make_random_move, make_move_alphabeta)
#minimax_alpha = Game.Game(make_move_minimax, make_move_alphabeta)

#random_minimemo = Game.Game(make_random_move, make_move_minimax_memoization)
#random_alphadeep = Game.Game(make_random_move, make_move_alphabeta_iterative_deepening)
#minimemo_alpha = Game.Game(make_move_minimax_memoization, make_move_alphabeta)
#minimemo_alphadeep = Game.Game(make_move_minimax_memoization, make_move_alphabeta_iterative_deepening)
#alphadeep_mini = Game.Game(make_move_alphabeta_iterative_deepening, make_move_minimax)


#mini_minimemo = Game.Game(make_move_minimax, make_move_minimax_memoization)
#alpha_alphadeep = Game.Game(make_move_alphabeta, make_move_alphabeta_iterative_deepening)

In [ ]:
game = Game.Game(make_move_alphabeta, make_random_move)
game.play()

In [ ]:
random_alpha.play()

In [ ]:
minimax_alpha.play()

In [ ]:
random_minimemo.play()

In [ ]:
random_alphadeep.play()

In [ ]:
minimemo_alpha.play()

In [ ]:
minimemo_alphadeep.play()

In [ ]:
alphadeep_mini.play()

In [ ]:
mini_minimemo.play()

In [ ]:
alpha_alphadeep.play()

In [ ]:
#print(game.move_times)
#white_d = sum(game.move_times[chess.WHITE]) / len(game.move_times[chess.WHITE])
#black_d = sum(game.move_times[chess.BLACK]) / len(game.move_times[chess.BLACK])

#print(white_d)
#print(black_d)

In [ ]:
game = Game.Game(make_move_alphabeta)
game.play()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d6ce9acd-52c5-4422-904d-8424da19408b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>